In [1]:
# Import tools
from splinter import Browser
import pandas as pd
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Create the temporary dataframe
temp_df = pd.DataFrame(columns =['ID', 'GPS_Code', 'name', 'continent', 'country', 'region', 'af_type', 'latitude', 'longitude', 'elevation', 'runway_L', 'runway_W', 'freq_count', 'airline_count'])

In [3]:
# Read HTML table
df1 = pd.read_html("https://airportdatabase.net/vanuatu/tanna-airport-tah_5012.html")[0]

In [4]:
# Rename columns for ease of reading
df1.columns = ['Col_1', 'Col_2']

In [5]:
# Create Name variable
df1.loc[0, 'Col_1'] = 'name'
df1['Col_2'] = df1['Col_2'].str.replace(' details and information', '')

In [6]:
# Clean text; remove "ft."
df1['Col_2'] = df1['Col_2'].str.replace(' ft.', '')

C:\Users\drchr\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [7]:
# Clean text; remove colons
df1['Col_1'] = df1['Col_1'].str.replace(':', '')

In [8]:
# Clean text; remove "iso"
df1['Col_1'] = df1['Col_1'].str.replace('iso ', '')

In [9]:
# Clean text; remove "ISO"
df1['Col_1'] = df1['Col_1'].str.replace('ISO ', '')

In [10]:
# Clean text; remove " Code"
df1['Col_1'] = df1['Col_1'].str.replace(' Code', '_Code')

In [11]:
# Remove null row
df2 = df1.dropna()

In [12]:
# count frequencies and navaids
freq_df = df2.loc[df2['Col_2'].str.contains("Mhz")]

In [13]:
# create freq_count variable
freq_count = len(freq_df)

In [14]:
# create name variable
name = df2.loc[df2['Col_1'].str.contains('name')]
name2 = name.iloc[0]['Col_2']

In [15]:
# create ID variable
ID = df2.loc[df2['Col_1'].str.contains('ident')]
ID2 = ID.iloc[0]['Col_2']

In [16]:
# create type variable
af_type = df2.loc[df2['Col_1'].str.contains('type')]
af_type2 = af_type.iloc[0]['Col_2']

In [17]:
# create latitude variable
latitude = df2.loc[df2['Col_1'].str.contains('latitude')]
latitude2 = latitude.iloc[0]['Col_2']

In [18]:
# create longitude variable
longitude = df2.loc[df2['Col_1'].str.contains('longitude')]
longitude2 = longitude.iloc[0]['Col_2']

In [19]:
# create elevation variable
elevation = df2.loc[df2['Col_1'].str.contains('elevation')]
elevation2 = elevation.iloc[0]['Col_2']

In [20]:
# create continent variable
continent = df2.loc[df2['Col_1'].str.contains('continent')]
continent2 = continent.iloc[0]['Col_2']

In [21]:
# create country variable
country = df2.loc[df2['Col_1'].str.contains('country')]
country2 = country.iloc[0]['Col_2']

In [22]:
# create region variable
region = df2.loc[df2['Col_1'].str.contains('Region')]
region2 = region.iloc[0]['Col_2']

In [23]:
# create GPS_code variable
GPS_Code = df2.loc[df2['Col_1'].str.contains('GPS_Code')]
GPS_Code2 = GPS_Code.iloc[0]['Col_2']

In [24]:
# find runway data rows
pattern = '[0-9]x[0-9]'
runway = df2['Col_2'].str.contains(pattern, na=False)

In [25]:
# tag runway data rows in table w/"True"
runway_B = pd.concat([df2, runway], axis=1)

In [26]:
# Assign column name to runway data tag
runway_B.columns = ['Col_1', 'Col_2', 'Col_3']

In [27]:
# Select runway data rows
runway_C = runway_B.loc[runway_B['Col_3'] == True]

In [28]:
# parse runway length/width variables
runway_C[['length', 'width']] = runway_C['Col_2'].str.split('x', expand=True)

C:\Users\drchr\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [29]:
# create runway Dataframe
runway_df = pd.DataFrame(runway_C, columns=['Col_1', 'Col_2', 'Col_3', 'length', 'width'])

In [30]:
# Convert runway length/width to integer datatype
runway_df['length'] = runway_df['length'].astype('int')
runway_df['width'] = runway_df['width'].astype('int')

In [31]:
# Create runway length/width totals
runway_df.loc['runway_total']= runway_df.sum(numeric_only=True, axis=0)

In [32]:
# Create total runway length variable
runway_L = runway_df.loc['runway_total']['length']

In [33]:
# Create total runway width variable
runway_W = runway_df.loc['runway_total']['width']

In [34]:
# Identify airlines data row
pattern = ' , '
airlines = df2['Col_2'].str.contains(pattern, na=False)

In [35]:
# tag airline data rows in table w/"True"
airlines_B = pd.concat([df2, airlines], axis=1)

In [36]:
# Assign column name to runway data tag
airlines_B.columns = ['Col_1', 'Col_2', 'Col_3']

In [37]:
# Select airline data row
airlines_C = airlines_B.loc[airlines_B['Col_3'] == True]

In [38]:
# Create airlines Dataframe
airlines_df = pd.DataFrame(airlines_C, columns=['Col_1', 'Col_2', 'Col_3'])

In [39]:
# Parse airlines data into rows
airlines_df2 = (airlines_df.set_index(['Col_3'])
   .apply(lambda x: x.str.split(',').explode())
   .reset_index())

In [40]:
# Tag row with null data
airlines_df2['Col_1'].replace('', np.nan, inplace=True)

In [41]:
# Delete row with null data
airlines_df2.dropna(subset=['Col_1'], inplace=True)

In [42]:
# Create airline_count variable
airline_count = len(airlines_df2)

In [43]:
# Write data to temp Dataframe
temp_df.at[0, 'ID'] = ID2
temp_df.at[0, 'GPS_Code'] = GPS_Code2
temp_df.at[0, 'name'] = name2
temp_df.at[0, 'continent'] = continent2
temp_df.at[0, 'country'] = country2
temp_df.at[0, 'region'] = region2
temp_df.at[0, 'af_type'] = af_type2
temp_df.at[0, 'latitude'] = latitude2
temp_df.at[0, 'longitude'] = longitude2
temp_df.at[0, 'elevation'] = elevation2
temp_df.at[0, 'runway_L'] = runway_L
temp_df.at[0, 'runway_W'] = runway_W
temp_df.at[0, 'freq_count'] = freq_count
temp_df.at[0, 'airline_count'] = airline_count
temp_df

,ID,GPS_Code,name,continent,country,region,af_type,latitude,longitude,elevation,runway_L,runway_W,freq_count,airline_count
0,NVVW,NVVW,Tanna Airport,Oceania (OC),Vanuatu (VU),Tafea (VU-TAE),medium airport,-19.45509910583496,169.22399902343750,19,4035.0,98.0,3,0


In [44]:
# Convert temp Dataframe to correct data types
temp_df = temp_df.astype({'latitude':'float', 'longitude':'float', 'elevation':'int', 'runway_L':'int', 'runway_W':'int', 'freq_count':'int', 'airline_count':'int'})

In [45]:
# Append to existing CSV file

temp_df.to_csv("Resources/AF1.csv", mode='a', index=False, header=False)